In [2]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
import json
from scipy import stats

### Домашнее задание 2. Интерпретация результатов АБ-теста

Срок сдачи: 31 марта 18:00

На мобильной версии Авито провели тест на добавление рекламного баннера на карточку товара между описанием и блоком рекомендаций.

Продуктовая гипотеза:
Добавление рекламного баннера будет приводить к увелечинию выручки без ущерба пользовательскому опыт.

<div>
<img src="attachment:screen_phone.png" width="300"/>
</div>

Задание:
1. Загрузите файл с данными, собранными за время аб теста.

2. На основе имеющихя в файле метрик формализуйте продуктовую гипотезу. Помимо количественных метрик рекомендую использовать ratio-метрики и метрики аггрегированные по пользователям(например, количество пользователей совершивших контакт) и при анализе теста обратите внимание на их чувтсвительность. (2 балла)

3.  Используя критерий Стьюдента, проинтерпретируйте как изменились средние выборок для выбранных вами метрик. Используйте уровень значимости 0.01.(2 балла)

4.  Выборка разделена на 200 бакетов. Примените к бакетам критерий Стьюдента и проинтерпретируйте как изменились средние выборок для выбранных вами метрик. Используйте уровень значимости 0.01. Сравните результаты с предыдущим пунктом. (2 балла)

6. Для анализа сформулированных в п.2 ratio-метрик воспольуйтесь Бутстрэпом. Являются ли эти метрики более чувствительными, чем базовые метрики? (2 балла)

7. Проинтерпретируйте результаты и сделайте выводы об изменениях для продукта. Рекомендуете ли вы изменение к запуску на 100% пользователей? (2 балла)

Формат данных

cookie_id - ид пользователя

split_group	- тестовая группа

searches - количество поисков

item_views - количество просмотров объявлений

contacts - количество контактов пользователя с продавцом	

rec_item_views - количество просмотров объявлений с рекомендацией

rec_contacts - контакты с рекомендацией

contact_cookie - пользователь, совершил контакт

advertising_clicks - клики на всю рекламу(Помимо добавленного баннера есть ещё баннеры в других местах. Эта метрика их учитывает.)

bucket - бакет

In [3]:
df = pd.read_csv('banner_test_results.csv')
df

,cookie_id,split_group,searches,item_views,contacts,rec_item_views,rec_contacts,contact_cookie,advertising_clicks,bucket
0,5898517031,test,0,21,7,0,0,1,0,31
1,28204005253,test,0,16,0,1,0,0,0,53
2,37285261090,test,0,4,0,0,0,0,0,90
3,48663044440,test,0,3,0,0,0,0,0,40
4,48835061540,test,0,2,0,0,0,0,0,140
...,...,...,...,...,...,...,...,...,...,...
1010800,3967387067306,test,0,44,0,0,0,0,1,106
1010801,3967407032924,control,0,5,0,1,0,0,1,124
1010802,3967507007864,control,40,166,7,30,1,1,1,64
1010803,3967529533807,control,2,10,0,1,0,0,1,7


2. На основе имеющихя в файле метрик формализуйте продуктовую гипотезу. Помимо количественных метрик рекомендую использовать ratio-метрики и метрики аггрегированные по пользователям(например, количество пользователей совершивших контакт) и при анализе теста обратите внимание на их чувтсвительность. (2 балла)

Добавление баннера должно повысить количество кликов на рекламу, что приведет к увеличению выручки. Я предлагаю использовать общее количество кликов на рекламу и ratio-метрику количество кликов на рекламу / количество просмотров страницы - какая доля из просмотров айтемов была сконвертирована в рекламный клик.

Это ключевые метрики, касающиеся первой части продуктовой гипотезы, про выручку. Для того, чтобы проследить, что пользовательский опыт не стал хуже, я бы обращала внимание на такие агрегированные метрики как количество пользователей, совершивших конртакт с продавцами; среднее количество просмотров (предполагается, что если нам удалось не ухудшить пользовательский опыт, то юзеры в среднем не станут реже просматривать объявления и контактировать с продавцами)


3.  Используя критерий Стьюдента, проинтерпретируйте как изменились средние выборок для выбранных вами метрик. Используйте уровень значимости 0.01.(2 балла)


Буду смотреть на изменение метрик:
- среднее количество кликов на рекламу
- количество юзеров, совершивших контакт с продавцами
- среднее количество просмотров объявлений

Гипотеза для среднего количества кликов на рекламу:
(1 - тест, 2 - контроль)

$H0: \mu_1 = \mu_2$ (среднее значение метрики на тесте и контроле равно)

$H1: \mu_1 \ge \mu_2$ (среднее значение метрики на тесте не меньше, чем на контроле)




Гипотеза для количества контактов с продавцами и среднего количества просмотров объявлений:
(1 - тест, 2 - контроль)

$H0: \mu_1 = \mu_2$ (среднее значение метрики на тесте и контроле равно)

$H1: \mu_1 \le \mu_2$ (среднее значение метрики на тесте не равно значению на контроле)

Здесь интерпретировать результаты следует так: если мы не отвергаем Н0, то пользовательский опыт в среднем не стал хуже. Если же мы принимаем альтернативную гипотезу, это значит, что при добавлении баннера данные пользовательские метрики в среднем просядут - то есть опыт пользователя ухудшится




In [19]:
clicks_test = df.loc[df.split_group == "test", 'advertising_clicks']
clicks_control = df.loc[df.split_group == "control", 'advertising_clicks']
print(np.var(clicks_control), np.var(clicks_test))

30.473171410830084 33.359079651980515


In [20]:
stats.ttest_ind(a=clicks_test, b=clicks_control, equal_var=True, alternative='greater')

Ttest_indResult(statistic=2.7769176008074683, pvalue=0.002743904963274941)

В данном случая, я считаю, что можно использовать вариант t-теста для выборок с равными дисперсиями. 

p-valuе < alpha, поэтому можно отвергнуть H0, и принять альтернативу: среднее количество кликов на рекламу, после добавления нового баннера не меньше, чем на контрольной группе



In [40]:
users_test = df.loc[df.split_group == "test", 'contacts']
users_control = df.loc[df.split_group == "control", 'contacts']
print(np.var(users_control), np.var(users_test))

91363.44974465488 141909.49673436253


In [41]:
stats.ttest_ind(a=users_test, b=users_control, equal_var=False, alternative='less')

Ttest_indResult(statistic=2.0728019166949747, pvalue=0.980904509084013)

p-value=0.98 > alpha=0.01, поэтому мы не отвергаем H0 и значит не можем утверждать, что среднее значение количества контактов после добавления баннера станет меньше, по сравнению с количеством контактов в контрольной группе 

In [42]:
views_test = df.loc[df.split_group == "test", 'item_views']
views_control = df.loc[df.split_group == "control", 'item_views']
print(np.var(views_control), np.var(views_test))

16662837.865840832 17619244.991101533


In [43]:
stats.ttest_ind(a=views_test, b=views_control, equal_var=False, alternative='less')

Ttest_indResult(statistic=0.5248938387491806, pvalue=0.7001714460471651)

p-value=0.7 > alpha=0.01, поэтому мы не отвергаем H0 и значит не можем утверждать, что среднее значение количества просмотров айтемов после добавления баннера станет меньше, по сравнению с количеством просмотров в контрольной группе 

В целом по текущим метрикам получается, что на уровне значимости 1%, добавление баннера действительно увеличит количество кликов на рекламу, но при этом ухудшения пользовательского опыта не произойдет. Наиболее чувствительной оказалась метрика "число кликов на баннер", что наверное хорошо, если мы занимаемся максимизацией их числа.

4.  Выборка разделена на 200 бакетов. Примените к бакетам критерий Стьюдента и проинтерпретируйте как изменились средние выборок для выбранных вами метрик. Используйте уровень значимости 0.01. Сравните результаты с предыдущим пунктом. (2 балла)

In [58]:
x = "advertising_clicks"
print("Бакеты, в которых мы можем отвергнуть Н0 и пронаблюдать увеличение средних кликов на рекламу:")
for i in range (0, 200):
  test_sample = df.loc[(df.split_group == "test") & (df.bucket == i), x]
  control_sample = df.loc[(df.split_group == "control") & (df.bucket == i), x]
  _, pvalue = stats.ttest_ind(a=test_sample, b=control_sample, equal_var=False, alternative='greater')
  if pvalue < 0.01:
    print("Номер бакета", i, " при p-value=",pvalue.round(5))

Бакеты, в которых мы можем отвергнуть Н0 и пронаблюдать увеличение средних кликов на рекламу:
Номер бакета 20  при p-value= 0.00141


In [59]:
x = "contacts"
print("Бакеты, в которых мы можем отвергнуть Н0 и пронаблюдать уменьшение среднего количества контактов с продавцами:")
for i in range (0, 200):
  test_sample = df.loc[(df.split_group == "test") & (df.bucket == i), x]
  control_sample = df.loc[(df.split_group == "control") & (df.bucket == i), x]
  _, pvalue = stats.ttest_ind(a=test_sample, b=control_sample, equal_var=False, alternative='less')
  if pvalue < 0.01:
    print("Номер бакета", i, " при p-value=",pvalue.round(5))

Бакеты, в которых мы можем отвергнуть Н0 и пронаблюдать уменьшение среднего количества контактов с продавцами:


In [166]:
x = "item_views"
print("Бакеты, в которых мы можем отвергнуть Н0 и пронаблюдать уменьшение среднего количества просмотров айтемов:")
for i in range (0, 200):
  test_sample = df.loc[(df.split_group == "test") & (df.bucket == i), x]
  control_sample = df.loc[(df.split_group == "control") & (df.bucket == i), x]
  _, pvalue = stats.ttest_ind(a=test_sample, b=control_sample, equal_var=False, alternative='less')
  if pvalue < 0.01:
    print("Номер бакета", i, " при p-value=",pvalue.round(5))

Бакеты, в которых мы можем отвергнуть Н0 и пронаблюдать уменьшение среднего количества просмотров айтемов:


5. Для анализа сформулированных в п.2 ratio-метрик воспольуйтесь Бутстрэпом. Являются ли эти метрики более чувствительными, чем базовые метрики? (2 балла)

In [159]:
def ratio(W):
  A = np.array(W).T.tolist()
  return np.mean(sum(A[0])/sum(A[1]))

In [160]:
def stat_intervals(stat, alpha = 0.01):
    return np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])

def get_bootstrap_samples(data, 
                              n_samples, 
                              custom_func = ratio,):
    return np.array([ratio(data[np.random.randint(0, len(data), len(data))]) for item in range(n_samples)])

In [162]:
test_sample = df.loc[(df.split_group == "test"), ['advertising_clicks', 'item_views']].to_numpy()
control_sample = df.loc[(df.split_group == "control"), ['advertising_clicks', 'item_views']].to_numpy()

In [163]:
b_test = get_bootstrap_samples(test_sample, 1000, ratio)

In [164]:
b_control = get_bootstrap_samples(control_sample, 1000, ratio)

In [165]:
stat_intervals(b_test-b_control, alpha = 0.01)

array([-1.99310290e-06,  8.02415914e-05])

Мне кажется, конверсионные метрики в целом должны быть более чувствительными, так как они by design показывают именно % совершения целевого действия и поэтому эффект (проводимое изменение) будет более точно оценен. Традиционные метрки менее конкретные.

При уровне значимости 1%, статистически значимого результата нет - доверительный интервал включает в себя 0, мы не отвергаем H0 и разница средних не задетектирована и разница в конверсии после теста может быть как больше 0, так и меньше.

6. Проинтерпретируйте результаты и сделайте выводы об изменениях для продукта. Рекомендуете ли вы изменение к запуску на 100% пользователей? (2 балла)

Мне кажется, что запускать продукт не следует. Единственный однозначно-зеленый тест это тест с метрикой "общее количество кликов на рекламу", однако при проведении аналогичного теста с разбиением на бакеты, существенный зеленый прокрас был только в 1 из 200 (я не делала повторного разбиения, а скорее всего именно в этот бакет попал сильный положительный выброс).

Тест с более чувствительной ratio-метрикой однозначно серый, отрицательные значения в его доверительный интервал входят, поэтому проводить такое изменение для всех точно не следует, хотя продуктовые метрики и не показывают красных результатов.